In [1]:
#https://www.analyticsvidhya.com/blog/2020/06/nlp-project-information-extraction/
    
from urllib.request import urlopen
from random import seed, randint
from src.preprocessing import PreProcessing
from src.sentenceregbr import SentenceRegBr
import os
from pathlib import Path
import pandas as pd
import codecs


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mbjesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# seed random number generator
seed(16)
BASE_DIR = Path(os.path.abspath('')).resolve()
objPreProcessing = PreProcessing()
objSentecesRegBr = SentenceRegBr()

  Abbreviation: [1.7426] art
  Abbreviation: [1.9803] art


### ProcessText

#### Insert DocBase

In [3]:
## Saving dataset
#dfBaseLegis = pd.read_json(Path(os.path.join(BASE_DIR,'dataset', 'legislacao.json')))
#dfBaseLegis.to_parquet(Path(os.path.join(BASE_DIR,'dataset', 'processed','legislacao_base.parquet')), engine='fastparquet')

In [4]:
dfBaseLegis = pd.read_parquet(Path(os.path.join(BASE_DIR,'dataset', 'processed','legislacao_base.parquet')), engine='fastparquet')

def __correctingIdDocBase(idDocBase):
    if(str(idDocBase) == 'L1309'):
        idDocBase =  'L13019' 
    return idDocBase

dfBaseLegis["id_doc_base"] = dfBaseLegis["id_doc_base"].apply(lambda x:__correctingIdDocBase(x))
dfBaseLegis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   path         35 non-null     object
 1   id_doc_base  35 non-null     object
 2   name         35 non-null     object
 3   esfera       35 non-null     object
 4   cat          35 non-null     object
 5   ementa       35 non-null     object
dtypes: object(6)
memory usage: 1.8+ KB


### Load text end sentences

#### Building the "third sector" dataset for expert evaluation


In [5]:
dfSentenca = pd.DataFrame()

for index,row in dfBaseLegis.iterrows():

    data_location = Path(os.path.join(BASE_DIR,'dataset','html',f'{row["path"]}'))
    text = objPreProcessing.getTextFromHTML(data_location)
    text = objPreProcessing.clearText(text[150:len(text)-300])
    sentenceList = objSentecesRegBr.getSentencesRegBrFormat(text,row["id_doc_base"],row["cat"])

    if row['cat'] == 'terceiro_setor':
        output_dict = [x['text_sentenca'] for x in sentenceList if x['cat_sentenca'] == 'terceiro_setor']

        if(len(output_dict)) > 0:
            with open(Path(os.path.join(BASE_DIR,'dataset','sentences','to_evaluate',f'{row["path"]}.txt')), 'w', encoding='utf-8') as infile:
                for line in output_dict:
                    infile.write(line)
                    infile.write('\n \n')
    else:
       dfSentenca = pd.concat([dfSentenca,pd.DataFrame(sentenceList)])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mbjesus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mbjesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
dfSentenca.head(2)

,id_sentenca,ordem,id_doc_base,tokens_total,tokens_no_stopwords,cat_sentenca,text_sentenca
0,L14133_SENT_9223,2,L14133,105,65,licitacoes,O PRESIDENTE DA REPÚBLICA FAÇO SABER QUE O CON...
1,L14133_SENT_9957,3,L14133,38,22,licitacoes,NÃO SÃO ABRANGIDAS POR ESTA LEI AS EMPRESAS PÚ...


In [7]:
dfSentenca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6200 entries, 0 to 729
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_sentenca          6200 non-null   object
 1   ordem                6200 non-null   int64 
 2   id_doc_base          6200 non-null   object
 3   tokens_total         6200 non-null   int64 
 4   tokens_no_stopwords  6200 non-null   int64 
 5   cat_sentenca         6200 non-null   object
 6   text_sentenca        6200 non-null   object
dtypes: int64(3), object(4)
memory usage: 387.5+ KB


#### Getting sentences evaluated by experts

In [8]:
pathFilesAlProc = Path(os.path.join(BASE_DIR,'dataset','sentences','already_evaluated'))
for root, subdirs, files in os.walk(pathFilesAlProc):
    for filename in files:
        id = filename.split('.')[0]
        with codecs.open(Path(os.path.join(pathFilesAlProc,filename)), encoding='utf-8') as f:
                sentences = [line.strip() for line in f.readlines() if line.strip() != '']

        for sentence in sentences:
            sentence = objPreProcessing.clearText(sentence)
            sentenceList = objSentecesRegBr.getSentencesRegBrFormat(sentence,id,"terceiro_setor")
 
            dfSentenca = pd.concat([dfSentenca,pd.DataFrame(sentenceList)])



In [9]:
dfSentenca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7312 entries, 0 to 3
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_sentenca          7312 non-null   object
 1   ordem                7312 non-null   int64 
 2   id_doc_base          7312 non-null   object
 3   tokens_total         7312 non-null   int64 
 4   tokens_no_stopwords  7312 non-null   int64 
 5   cat_sentenca         7312 non-null   object
 6   text_sentenca        7312 non-null   object
dtypes: int64(3), object(4)
memory usage: 457.0+ KB


In [10]:
dfBaseLegis = pd.merge(dfBaseLegis,dfSentenca,on="id_doc_base", how="left")

In [11]:
dfBaseLegis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7498 entries, 0 to 7497
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   path                 7498 non-null   object
 1   id_doc_base          7498 non-null   object
 2   name                 7498 non-null   object
 3   esfera               7498 non-null   object
 4   cat                  7498 non-null   object
 5   ementa               7498 non-null   object
 6   id_sentenca          7498 non-null   object
 7   ordem                7498 non-null   int64 
 8   tokens_total         7498 non-null   int64 
 9   tokens_no_stopwords  7498 non-null   int64 
 10  cat_sentenca         7498 non-null   object
 11  text_sentenca        7498 non-null   object
dtypes: int64(3), object(9)
memory usage: 761.5+ KB


In [12]:
dfBaseLegis.head(3)

,path,id_doc_base,name,esfera,cat,ementa,id_sentenca,ordem,tokens_total,tokens_no_stopwords,cat_sentenca,text_sentenca
0,L13019.htm,L13019,"LEI 13.019, DE 31 DE JULHO DE 2014",federal,terceiro_setor,Estabelece o regime jurídico das parcerias ent...,L13019_SENT_5181,1,85,48,terceiro_setor,ESTABELECE O REGIME JURÍDICO DAS PARCERIAS VOL...
1,L13019.htm,L13019,"LEI 13.019, DE 31 DE JULHO DE 2014",federal,terceiro_setor,Estabelece o regime jurídico das parcerias ent...,L13019_SENT_792,1,99,55,terceiro_setor,ESTABELECE O REGIME JURÍDICO DAS PARCERIAS ENT...
2,L13019.htm,L13019,"LEI 13.019, DE 31 DE JULHO DE 2014",federal,terceiro_setor,Estabelece o regime jurídico das parcerias ent...,L13019_SENT_6932,1,86,51,terceiro_setor,ESTA LEI INSTITUI NORMAS GERAIS PARA AS PARCER...


In [13]:
## Saving dataset
dfBaseLegis.to_parquet(Path(os.path.join(BASE_DIR,'dataset', 'processed','legislacao_base_sentencas.parquet')), engine='fastparquet')
dfBaseLegis.to_excel(Path(os.path.join(BASE_DIR,'dataset', 'processed','legislacao_base_sentencas.xlsx')), index=False)